In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join, QTable, vstack
import astropy.units as u
import sys
import pyneb as pn
from multiprocessing import Pool
import multiprocessing as mp
import math
from astropy.io import fits
from orcs.process import SpectralCube

from astropy.nddata import NDData, Cutout2D
from astropy.wcs import WCS

import astropy.units as u
from astropy.coordinates import SkyCoord

from reproject import reproject_interp
from regions import PixCoord

import pylab as pl

### Choose a Galaxy to Analyze

In [ ]:
galaxynum = 4
galdic = {1:'NGC4254', 2:'NGC4535', 3:'NGC3351', 4:'NGC2835', 5:'NGC0628'}  #There is no SITELLE data for NGC 4254, NGC 2835 has the best data 
galaxy = galdic[galaxynum]
galaxy

# Import Data

### The data obtained from SITELLE is contained in the `name_cube.hdf5` file

In [ ]:
infile = f"/home/habjan/jupfiles/data/{galaxy}_cube.hdf5"
cube = SpectralCube(infile)

### Import muse image for reprojection

In [ ]:
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_IMAGE_FOV_Johnson_B_WCS_Pall_mad.fits")
muse_data = NDData(data=hdul['DATA'].data, mask=np.isnan(hdul['DATA'].data), meta=hdul['DATA'].header, wcs=WCS(hdul['DATA'].header))    
muse_data.data[muse_data.data==0] = np.nan

### Import Halpha Flux values for visualizaton (This file is not needed for the analysis)

In [ ]:
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_MAPS.fits")
Halpha = NDData(data=hdul['HA6562_FLUX'].data, mask=np.isnan(hdul['HA6562_FLUX'].data), meta=hdul['HA6562_FLUX'].header, wcs=WCS(hdul['HA6562_FLUX'].header))
Halpha.data[muse_data.data==0] = np.nan

### Import HII region spatial masks for the location of each HII region

In [ ]:
# Import the HII region spatial masks
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_nebulae_mask_V2.fits")
nebulae_mask = NDData(data = hdul[0].data.astype(float), mask=Halpha.mask, meta=hdul[0].header, wcs=WCS(hdul[0].header)) 
nebulae_mask.data[nebulae_mask.data==-1] = np.nan

### WCS information from the 'name_cube.fits' file can be obtained to correct the WCS information in the 'name_deepframe.fits' file

In [ ]:
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_cube.fits")
header = hdul[0].header
wcs = WCS(header,naxis=2)

### The 'name_deepframe.fits' file is read in to get the correct 2d header

In [ ]:
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_deepframe.fits")
deepframe = NDData(data=hdul[0].data, meta=hdul[0].header, wcs=wcs)

### Import PHANGS Muse Nebular catalog

In [ ]:
infile = open("/home/habjan/jupfiles/data/Nebulae_catalogue_v3.fits",'rb')
hdul = Table.read(infile)
musedata = hdul[hdul['gal_name'] == f'{galaxy}']

# Analyze data to obtain Spectra

### Plot some of the data from above for visualization

In [ ]:
imagenamedic = {1:f'{galaxy}_IMAGE_FOV_Johnson', 2:f'{galaxy}_MAPS', 3:f'{galaxy}_nebulae_mask_V2', 4:f'{galaxy}_deepframe'}
plotdatadic = {1:muse_data, 2:Halpha, 3:nebulae_mask, 4:deepframe}

imnum1 = 3
imagename1 = imagenamedic[imnum1]
plotdata1 = plotdatadic[imnum1]


ax1 = plt.subplot(1,2,1, projection=wcs)
ax1.imshow(plotdata1.data, origin='lower', vmin=-100., vmax=2000.)
ax1.coords['ra'].set_axislabel('Right Ascension')
ax1.coords['dec'].set_axislabel('Declination')
ax1.set_title(imagename1)

imnum2 = 2
imagename2 = imagenamedic[imnum2]
plotdata2 = plotdatadic[imnum2]

ax2 = plt.subplot(1,2,2, projection=wcs)
ax2.imshow(plotdata2.data, origin='lower', vmin=-100, vmax=2000)
ax2.coords['ra'].set_axislabel('Right Ascension')
ax2.coords['dec'].set_axislabel('Declination')
ax2.coords['dec'].set_axislabel_position('r')
ax2.coords['dec'].set_ticklabel_position('r')
ax2.set_title(imagename2)

### Using `reporject_interp` we can visualize what the MUSE image looks like inside the SITELLE image

In [ ]:
array, footprint = reproject_interp(nebulae_mask, output_projection=wcs, shape_out=deepframe.data.shape)

### Plot the reprojected image and footprint of the MUSE data in 'SITELLE space'

In [ ]:
ax1 = plt.subplot(1,2,1, projection=wcs)
ax1.imshow(array, origin='lower', vmin=-2.e-4, vmax=5.e-4)
ax1.coords['ra'].set_axislabel('Right Ascension')
ax1.coords['dec'].set_axislabel('Declination')
ax1.set_title('Reprojected MUSE image \n in to SITELLE')

ax2 = plt.subplot(1,2,2, projection=wcs)
ax2.imshow(footprint, origin='lower', vmin=0, vmax=1.5)
ax2.coords['ra'].set_axislabel('Right Ascension')
ax2.coords['dec'].set_axislabel('Declination')
ax2.coords['dec'].set_axislabel_position('r')
ax2.coords['dec'].set_ticklabel_position('r')
ax2.set_title('footprint of MUSE in SITELLE')

### Plot the SITELLE image

In [ ]:
deep = cube.get_deep_frame()
deep.imshow(perc=95, wcs=True)
pl.grid()
cb = pl.colorbar(shrink=0.8, format='%.1e')
cb.ax.set_title('counts')
#deep.to_fits('deep_frame.fits')

### Use the SITELLE deepframe to reproject the MUSE HII region mask

In [ ]:
array, footprint = reproject_interp(nebulae_mask, output_projection=wcs, shape_out=deepframe.data.shape) #output_projection=wcs, shape_out=(223, 899, 900))
repro_mask = NDData(data=array, meta=nebulae_mask.meta, wcs=wcs)

### Plot the reprojected HII region mask

In [ ]:
ax1 = plt.subplot(1,2,1, projection=WCS(repro_mask.meta))
ax1.imshow(array, origin='lower', vmin=-2.e-4, vmax=5.e-4)
ax1.coords['ra'].set_axislabel('Right Ascension')
ax1.coords['dec'].set_axislabel('Declination')
ax1.set_title('Reprojected MUSE image \n in to SITELLE')

ax2 = plt.subplot(1,2,2, projection=WCS(repro_mask.meta))
ax2.imshow(footprint, origin='lower', vmin=0, vmax=1.5)
ax2.coords['ra'].set_axislabel('Right Ascension')
ax2.coords['dec'].set_axislabel('Declination')
ax2.coords['dec'].set_axislabel_position('r')
ax2.coords['dec'].set_ticklabel_position('r')
ax2.set_title('footprint of MUSE in SITELLE')

### Find the end points of the MUSE data within the SITELLE data

In [ ]:
xvalues = np.where(np.any(footprint,axis=0))
xmin,xmax = np.min(xvalues),np.max(xvalues)
yvalues = np.where(np.any(footprint,axis=1))
ymin,ymax = np.min(yvalues),np.max(yvalues)

### Plot the SITELLE data within the bounds of the MUSE data

In [ ]:
deep = cube.get_deep_frame()
deep.imshow(perc=95, wcs=wcs)
pl.grid()
#pl.xlim(xmin,xmax)
#pl.ylim(ymin,ymax)
pl.title(f'{galaxy} SITELLE deepframe')# \n in MUSE bounds')
pl.xlabel('R.A.')
pl.ylabel('Dec.')
cb = pl.colorbar(shrink=0.8, format='%.1e')
cb.ax.set_title('Integrated \nFlux')

### Reproject the HII region mask to the same image size as the SITELLE image

In [ ]:
array, footprint = reproject_interp(nebulae_mask, output_projection=wcs, shape_out=deepframe.data.shape)
repro_mask = NDData(data=array, meta = nebulae_mask.meta, wcs=wcs)

### Overlay the HII region locations onto the SITELLE image

In [ ]:
deep = cube.get_deep_frame()
#deep.imshow(perc=95, wcs=wcs, cmap='Greys', label='SITELLE deep frame')
deep.imshow(wcs=wcs, cmap='Greys')

plt.imshow(repro_mask.data, cmap='Reds')#,  alpha=0.9)
pl.title(f'{galaxy} SITELLE deepframe \n overlaid with HII region masks')
pl.gca().invert_xaxis()
pl.xlabel('R.A.')
pl.ylabel('Dec.')

for i in np.array([13, 767, 915]):
    testreg = i
    testpix = np.where(repro_mask.data == testreg)
    pl.scatter(testpix[1], testpix[0], marker='o', alpha=0.9, c='purple')#, label='HII region')

#testreg = 767
#testpix = np.where(repro_mask.data == testreg)
#pl.scatter(testpix[1], testpix[0], marker='.', alpha=0.05, c='purple')

zoomval = 0
pl.xlim(xmin*(1-zoomval),xmax*(1+zoomval))
pl.ylim(ymin*(1-zoomval),ymax*(1+zoomval))
#cb = pl.colorbar(shrink=0.8, format='%.1e')
#cb.ax.set_title('Integrated \nFlux')

### Find the brightest regions in the galaxy

In [ ]:
np.where(musedata['OII7319_FLUX'] > 30*musedata['OII7319_FLUX_ERR'])

### Plot a spectrum for a given pixel region

In [ ]:
regnum = 13

galveldic = {'NGC4254': 2388 , 'NGC4535': 1954  , 'NGC3351': 775, 'NGC2835': 867, 'NGC0628':651}
galvel = galveldic[galaxy]

wave1 = 3726
wave2 = 3729

w1 = wave1*(musedata[regnum]['HA6562_VEL']+galvel)/(299792) + wave1
w2 = wave2*(musedata[regnum]['HA6562_VEL']+galvel)/(299792) + wave2

reg = np.where(repro_mask.data == regnum)
reg = (reg[1], reg[0])

spectrum1 = cube.extract_integrated_spectrum(reg, median=True);
#spectrum2 = cube.extract_integrated_spectrum(reg, mean_flux=True)

fac = cube.params.flambda/cube.dimz/cube.params.exposure_time
#fac = 10**-20

fig = plt.figure()
fig.set_size_inches(10, 4)

#flux = np.real(spectrum1[1])/(fac)
flux = spectrum1[1]

wave = (1/spectrum1[0])*10**8

plt.plot(wave, flux, c='k', label='Spectrum')
plt.axvline(w1, linestyle='--', c='r', alpha=0.75, label=f'[OII]{wave1} at {round(w1,1)} Angstrom')
plt.axvline(w2, linestyle='--', c='orange', alpha=0.75, label=f'[OII]{wave2} at {round(w2,1)} Angstrom')
plt.xlim(wave1-150, wave1+150)

plt.xlabel('Wavelength [Angstrom]')
plt.ylabel(r'Flux [$\frac{erg}{cm^{2} \cdot s \cdot Angstrom}$]')
plt.title(f'Integrated spectrum of Region {regnum} in {galaxy}')
plt.legend(loc='lower right')

### The line of code below finds the MUSE nebulae mask pixels and converts them into degree coordinates

In [ ]:
regnum = 767
testcoord = WCS(nebulae_mask.meta).pixel_to_world(np.where(nebulae_mask.data == regnum)[1], np.where(nebulae_mask.data == regnum)[0])

### The code below takes the degree coordinates from above and converts them into SITELLE image pixel coordinates 

In [ ]:
sitpix = np.transpose(cube.world2pix((testcoord.ra.degree, testcoord.dec.degree)))
sitpix = sitpix[0].astype(np.int64), sitpix[1].astype(np.int64)

### Plot all of the regions on the plot below using the obtained SITELLE pixels

In [ ]:
deep = cube.get_deep_frame()
deep.imshow(wcs=wcs)#, cmap='Greys')


pl.title(f'{galaxy} SITELLE deepframe \n overlaid with HII region pixels')
pl.xlabel('R.A.')
pl.ylabel('Dec.')

for i in range(30):
    regnum = i
    testcoord = WCS(nebulae_mask.meta).pixel_to_world(np.where(nebulae_mask.data == regnum)[1], np.where(nebulae_mask.data == regnum)[0])
    sitpix = np.transpose(cube.world2pix((testcoord.ra.degree, testcoord.dec.degree)))
    sitpix = sitpix[0].astype(np.int64), sitpix[1].astype(np.int64)
    pl.scatter(sitpix[0], sitpix[1], marker='.', s=100, alpha=0.002, c='purple')

zoomval = 0.1
pl.xlim(xmin*(1-zoomval),xmax*(1+zoomval))
pl.ylim(ymin*(1-zoomval),ymax*(1+zoomval))